In [1]:
import glob
import time
import numpy as np
import cv2 as cv
import copy
from sklearn.cluster import MeanShift

In [2]:
occupancy_count = 0
object_states = {}

def Tracking(img_address):
    global occupancy_count
    global object_states
    
    img_row_size_org = 670
    img_col_size_org = 480
    img_row_size = 67
    img_col_size = 48
    img_size = img_row_size*img_col_size
    img_min_x = 15; img_max_x = 40
    img_min_y = 0; img_max_y = 50
    bandwidth = 6.5
    
    last_processed_file_name = None # 마지막으로 처리된 파일 이름을 저장하는 변수 추가

    frames = {}; frames[0] = []
    similar_boxes = []

    while True:
        # 매 루프마다 현재 디렉토리의 모든 파일을 정렬하여 가져옵니다.
        all_files = sorted(glob.glob(img_address))
        
        # 처리할 새로운 파일 목록을 필터링합니다.
        new_files_to_process = []
        if last_processed_file_name:
            for file in all_files:
                if file > last_processed_file_name:
                    new_files_to_process.append(file)
        else:
            # 첫 실행 시에는 모든 파일을 처리합니다.
            new_files_to_process = all_files
        
        if new_files_to_process:
            for f in range(len(new_files_to_process)):
                current_file = new_files_to_process[f]
            
                img_real = []
                img_data = []
                lower_orange = np.array([10, 100, 100])
                upper_orange = np.array([20, 255, 255])
                
                img_black = cv.imread(current_file, cv.IMREAD_GRAYSCALE)
                img_color = cv.imread(current_file, cv.IMREAD_COLOR)
                if img_color is None:
                    continue

                img_rgb = cv.resize(cv.cvtColor(img_color, cv.COLOR_BGR2RGB), (img_col_size, img_row_size))[img_min_y:img_max_y,img_min_x:img_max_x,:]
                img_hsv = cv.resize(cv.cvtColor(img_color, cv.COLOR_BGR2HSV), (img_col_size, img_row_size))[img_min_y:img_max_y,img_min_x:img_max_x,:]
                mask = cv.inRange(img_hsv, lower_orange, upper_orange)
                output_image = np.zeros((img_hsv.shape[0], img_hsv.shape[1]), dtype=np.uint8)
                output_image[mask > 0] = 255
            
                img_real += [cv.resize(cv.cvtColor(img_color, cv.COLOR_BGR2RGB), (img_col_size_org, img_row_size_org))]
                img_data += [output_image]
                
                # Dotting
                X = []
                for i in range(len(img_data[0])):
                    for j in range(len(img_data[0][0])):
                        for k in range(int((img_data[0][i][j])/250)):
                            X += [[j,i]]
                
                if len(X)<int(bandwidth):
                    X += [[0,0]]*int(bandwidth)
                X = np.array(X)
        
                # Mean Shift Clustering
                ms = MeanShift(bandwidth=bandwidth, bin_seeding=True)
                ms.fit(X)
                
                # Finding the Cluster Centers
                cluster_centers = ms.cluster_centers_
                labels = ms.labels_
            
                # Finding Outliers
                distances = np.linalg.norm(X - cluster_centers[labels], axis=1)
                threshold_factor = 1.8
                mean_distances = np.array([np.mean(distances[labels == i]) for i in range(len(cluster_centers))])
                thresholds = threshold_factor * mean_distances[labels]
                is_outlier = distances.flatten() > thresholds
                X = X[~is_outlier]
                labels = labels[~is_outlier]
                
                # Finding the Number of Clusters
                n_clusters_ = len(np.unique(labels))
                
                # Bounding Boxes
                bounding_boxes = []
                for label in np.unique(labels):
                    cluster_points = X[labels == label]
                    min_x, min_y = cluster_points.min(axis=0)
                    max_x, max_y = cluster_points.max(axis=0)
                    width = max_x - min_x
                    height = max_y - min_y
                    bounding_box_size = (max_x - min_x + 1)*(max_y - min_y + 1)
                    cluster_probabilities = len(cluster_points)/bounding_box_size * len(cluster_points)/len(X)
                    bounding_box = [img_min_x+min_x, img_min_y+min_y, img_min_x+max_x, img_min_y+max_y, np.round(cluster_probabilities,2)]
                    bounding_boxes.append(bounding_box)
                    if cluster_probabilities>=0.1 and len(cluster_points)>10:
                        pass
                frames[f+1] = bounding_boxes
        
                # Tracking
                current_boxes = frames[f+1]
                previous_boxes = frames.get(f, [])  # frames[f]는 잘못된 인덱스일 수 있음.
            
                previous_similar_boxes = copy.deepcopy(similar_boxes)
                for i, current_box in enumerate(current_boxes):
                    if len(X)>15 and current_box[3]>=img_max_y-13 and current_box[4]>=0.15:
                        similar_boxes_last = [j[-1] for j in similar_boxes]
                        current_new = 1
                        for j, bbox in enumerate(similar_boxes_last):
                            try:
                                if abs(bbox[0] - current_box[0]) <= 4 or abs(bbox[2] - current_box[2]) <= 4:
                                    similar_boxes[j] += [current_box]
                                    current_new = 0
                            except:
                                current_new = 1
                        if current_new==1:
                            similar_boxes += [[current_box]]
        
                for i in range(len(previous_similar_boxes)):
                    if previous_similar_boxes[i] == similar_boxes[i]:
                        try:
                            prev_height = similar_boxes[i][-2][3] - similar_boxes[i][-2][1]
                            current_height = similar_boxes[i][-1][3] - similar_boxes[i][-1][1]
                            if current_height >= prev_height:
                                similar_boxes[i] = "IN"
                            elif current_height < prev_height:
                                similar_boxes[i] = "OUT"
                        except:
                            if similar_boxes[i] != "IN" and similar_boxes[i] != "OUT":
                                similar_boxes[i] = "NONE"
        
                # Counting logic
                for i in range(len(similar_boxes)):
                    current_state = similar_boxes[i]
                    # IN event: check if state changed from something else to 'IN'
                    if current_state == 'IN' and object_states.get(i) != 'IN':
                        occupancy_count += 1
                        object_states[i] = 'IN'
                    # OUT event: check if state changed from something else to 'OUT'
                    elif current_state == 'OUT' and object_states.get(i) != 'OUT':
                        occupancy_count -= 1
                        object_states[i] = 'OUT'
                    # Reset state for objects that are no longer IN/OUT
                    elif current_state == 'NONE' and object_states.get(i) in ['IN', 'OUT']:
                        del object_states[i]
                    # Update the state if it's 'DETECT' to prevent re-counting
                    elif current_state == 'DETECT':
                        object_states[i] = 'DETECT'
                        
                # Print results
                result_text = []
                for i in range(len(similar_boxes)):
                    if similar_boxes[i] == 'IN' or similar_boxes[i] == 'OUT':
                        result_text += [f"Object {i}: {similar_boxes[i]}\n"]
                    elif similar_boxes[i] != 'NONE':
                        result_text += [f"Object {i}: DETECT\n"]
                result_texts = "\n".join(result_text)
                #print(f"--- Frame {f+1} Results ---")
                #print(result_texts)
                print(f"Occupancy Count: {occupancy_count}")
        
        if new_files_to_process:
            last_processed_file_name = new_files_to_process[-1]
        
        time.sleep(1)

In [3]:
Tracking("all_class/02. Multiple OUT/*.png")

Occupancy Count: 0
Occupancy Count: 0
Occupancy Count: 0
Occupancy Count: -1
Occupancy Count: -2
Occupancy Count: -2
Occupancy Count: -2
Occupancy Count: -2
Occupancy Count: -2
Occupancy Count: -2


KeyboardInterrupt: 